## Usando notebook de Eugenio

In [ ]:
!pip install imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import datetime
from imblearn.under_sampling import RandomUnderSampler
from sklearn.impute import SimpleImputer

# from lightgbm import LGBMClassifier
import lightgbm as lgbm

#######
# rutas
# datasets
from config import dataset_file_fe6_6xxpqt, dataset_file_fe6_6xxpqt_under # con lag1&2 + delta1&2
      
# optimizacion
from config import db_path
# modelos
from config import modelos_path
# predicciones
from config import pred_path


In [ ]:

ganancia_acierto = 273000
costo_estimulo = 7000

semillas = [437809, 327347, 392879, 455783, 217163]

In [ ]:
data = pd.read_parquet(dataset_file_fe6_6xxpqt)

## Undersampleo con bajas unificadas

In [ ]:
# unificación de bajas
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

# quitando meses sin clase 
meses_excluidos = [202108, 202109] # meses con clase ternaria incompleta
data = data[~data['foto_mes'].isin(meses_excluidos)]
data['foto_mes'].unique()

In [ ]:
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    
    # voy a incrementar esa proporcion por 10

    estrategia={0: int(len(y[y == 0]) * 0.02), 
                1: len(y[y == 1])}

    print(f"Se retienen {estrategia[0]} de la clase mayoritaria y {estrategia[1]} de la minoritaria")
    
    # new_proportion = minority_proportion * 10

    rus = RandomUnderSampler(sampling_strategy=estrategia, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rearmar
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes

    df_subsampled.append(group_resampled)

# Mergear
data = pd.concat(df_subsampled, ignore_index=True)

data.shape

Guardo por las dudas

In [ ]:
data.to_parquet(dataset_file_fe6_6xxpqt_under, index=False)

## Entrenamiento final

In [4]:
# para recordar los periodos en los que entrenamos el modelo final:
data['foto_mes'].unique()

array([201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
       202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
       202101, 202102, 202103, 202104, 202105, 202106, 202107])

In [5]:
data.shape

(112198, 764)

### Preprocesamiento

In [6]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [7]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)

# Imputacion de Xs
cols_with_all_nan = X_train.columns[X_train.isna().all()].tolist()
print("Columns with all NaN values:", cols_with_all_nan)
X_train = X_train.drop(columns=cols_with_all_nan)

# Imputación de nulls
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_imp = pd.DataFrame(imp_median.fit_transform(X_train), columns=X_train.columns)

# Codificar variables categóricas
categorical_features = [col for col in X_train_imp.columns if X_train_imp[col].dtype == 'object']

# Convertir variables categóricas a 'category' dtype para LightGBM
for col in categorical_features:
    X_train_imp[col] = X_train_imp[col].astype('category')

y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

In [9]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

### Mejores parametros

Eugenio

In [ ]:
params = {'num_leaves': 470,
            'learning_rate': 0.006845251039139201,
            'min_data_in_leaf': 305,
            'feature_fraction': 0.4104147707505324,
            'bagging_fraction': 0.1016227845120685}

params.update({'n_jobs': -1,
                'objective': 'binary',
                'boosting_type': 'gbdt',
                'first_metric_only': True,
                'boost_from_average': True,
                'feature_pre_filter': False,
                'max_bin': 31,
               })

print(params)


### Datos a predecir para Kaggle

In [ ]:
# vuelvo a leer sin undersamplear
data_ = pd.read_parquet(dataset_file_fe6_6xxpqt)

mes_test = 202109

X_kaggle = data_[data_['foto_mes'] == mes_test]
X_kaggle = X_kaggle.drop(columns=['clase_ternaria']) # nulls

del data_

# prepro en X:kaggle
X_kaggle = X_kaggle.drop(columns=cols_with_all_nan)
X_kaggle_imp = pd.DataFrame(imp_median.transform(X_kaggle), columns=X_train.columns)
for col in categorical_features:
    X_kaggle_imp[col] = X_kaggle_imp[col].astype('category')

numero_de_cliente = X_kaggle_imp['numero_de_cliente'].astype(int)


### Entranmiento con semillas

In [ ]:
print("Running semillerío para entrega")
print(params)

# para registrar las probabilidades
df_sem_proba = pd.DataFrame({
                            'client': numero_de_cliente.values,
                        })

j = 0
s_r = range(217163, 455783, 7*7*7*7*3) # 34 semillas
s_r = list(range(2)) # 2 semillas de prueba

s_total = len(list(s_r))
for s in s_r:
    # nueva instancia del modelos con semilla
    seed = s + (7+j)
    print(f"Entrenando modelo con semilla: {seed}, {j+1} de {s_total}")
    # model = LGBMClassifier(**params, random_state=seed)
    # # entreno
    # model.fit(X=X_train_imp, y=y_train_binaria)

    # seteo semilla
    params.update({'seed': seed})
    # training set
    train_data = lgbm.Dataset(X_train_imp,
                              label=y_train_binaria,
                              weight=w_train)
    model = lgbm.train(params,
                       train_data,
                       num_boost_round=1509) # best iteration de opt
    
    # predigo proba
    y_pred_proba = model.predict_proba(X_kaggle_imp)
    # proba baja+2
    # proba_baja2 = y_pred_proba[:,2]
    # proba baja
    proba_baja = y_pred_proba[:,1]
    df_sem_proba[f'proba_s{seed}'] = proba_baja
    j += 1

# Promediando proba de cada semilla
proba_s_columns = df_sem_proba.filter(regex='^proba_s')
proba_s_mean = proba_s_columns.mean(axis=1)

df_sem_proba['proba_sem_mean'] = proba_s_mean

# Umbral
thr_opt_sem = 0.0195 # segun comp03_kaggle_api_sub

# Segun Denicolay, el óptimo ronda los 11 mil estímulos

# Prediccion
df_sem_proba['pred'] = np.where(df_sem_proba.proba_sem_mean >= thr_opt_sem, 1, 0)

df_sem_proba.head()

In [ ]:
submission = pd.DataFrame({
    'numero_de_cliente': numero_de_cliente.values,
    'Predicted': df_sem_proba['pred'].values
})

# Imprimir value counts de las predicciones
value_counts = submission['Predicted'].value_counts()
total_count = len(submission)
print("\nValue Counts:")
print(value_counts)
print("\nFrecuencia Relativa:")
print((value_counts / total_count) * 100)

submission.info()

In [ ]:
ft = "%dT-%m-%Y%H-%M-%S"
t_now = datetime.datetime.now().strftime(ft)

pred_name = f"pred_sem_03_eug_sem{s_total}_"+t_now+".csv"

proba_file = pred_path + "probas/" + pred_name
pred_file = pred_path + pred_name

# Guardamos las probas
df_sem_proba.to_csv(proba_file, index=False)
print(f"Probas guardadas en {proba_file}")

# Guardar el DataFrame en un archivo CSV
submission.to_csv(pred_file, index=False)
print(f"Predicciones guardadas en {pred_file}")